In [ ]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tokenize
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow as tf
print(tf.__version__)

2.6.0


In [ ]:
!pip install tensorflow==1.14.0
!pip install tensorflow_datasets==2.0|
import tensorflow as tf
print(tf.__version__)


     |████████████████████████████████| 109.3 MB 30 kB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
     |████████████████████████████████| 3.1 MB 54.7 MB/s 
     |████████████████████████████████| 488 kB 52.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you

/bin/bash: -c: line 1: syntax error: unexpected end of file
2.6.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
import keras
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint

In [ ]:
# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [ ]:
dataset, information = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteHSGN96/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteHSGN96/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load the dataset but only keep the top n words, zero the rest
imdb = keras.datasets.imdb
top_words=5000
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=top_words)
print(X_train[0])
word_to_id = keras.datasets.imdb.get_word_index()
id_to_word = {value:key for key,value in word_to_id.items()}
for id in X_train[0]:
  print("********************************************")
  print(id)
  print(' '.join(id_to_word.get(id - 3, '?')))

17473536/17464789 [==============================] - 0s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1654784/16

In [ ]:
# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(LSTM(70))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
conv1d (Conv1D)              (None, 496, 64)           10304     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 124, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 124, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 70)                37800     
_________________________________________________________________
dense (Dense)                (None, 1)                 71        
Total params: 208,175
Trainable params: 208,175
Non-trainable params: 0
__________________________________________________

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.44%


In [ ]:
import keras
from keras.preprocessing.text import Tokenizer
def conv_to_proper_format(sentence):
  word_index=imdb.get_word_index()
  sentences=tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ',char_level=False, oov_token=None,
    document_count=0)
  print(sentences)
  sentence=np.array([word_index[word] if word in word_index else 0 for word in sentence])#Encoding into sequence of integers
  sentence[sentence>5000]=2
  L=500-len(sentence)
  sentence=np.pad(sentence, (L,0), "constant")
  sentence=sentence.reshape(1,-1)
  #print(sentence)
  return sentence

In [ ]:
r1=conv_to_proper_format("bad movie")
model.predict(r1)

array([[0.63842624]], dtype=float32)

In [ ]:
vec=imdb.get_word_index()
def get_senti(sent):
    m=494
    x=np.zeros((1,500))
    for i in sent.split():
      x[0][m]=(vec[i]+3)
      m=m+1
      #print(x)
    return x

r1=get_senti("this movie could have been better")
model.predict(r1)


array([[0.332477]], dtype=float32)

In [ ]:
import keras
def conv_to_proper_format(sentence):
  word_index=imdb.get_word_index()
  sentences=tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ',char_level=False, oov_token=None,
    document_count=0)
  print(sentences)
  sentence=np.array([word_index[word] if word in word_index else 0 for word in sentence])#Encoding into sequence of integers
  #Encoding into sequence of integers
  sentence[sentence>5000]=2
  L=500-len(sentence)
  sentence=np.pad(sentence, (L,0), "constant")
  sentence=sentence.reshape(1,-1)
  #print(sentence)
  return sentence

In [ ]:
r1=conv_to_proper_format(["this", "was", "a" ,"good" ,"movie"])
model.predict(r1)

array([[0.5075164]], dtype=float32)

In [ ]:
print("Please Enter your valuable review")
gt=input()
gtg=["good","bad","random"]
#gtg=[gts]


texts = gt
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X_tests = tokenizer.texts_to_sequences(texts)


X_tests = pad_sequences(X_tests, maxlen=max_review_length)

y_pred=model.predict(X_tests)
classes_x=np.argmax(y_pred,axis=1)


#if(y_pred==1):
  #y_pred=0
#if(y_pred>1):
  #y_pred=1
print(gtg)
#print(y_pred)
#return np.argmax(y_pred)
print(y_pred)

Please Enter your valuable review
This is a good movie
['good', 'bad', 'random']
[[0.50346327]
 [0.5034425 ]
 [0.50332224]
 [0.50328326]
 [0.5031633 ]
 [0.50332224]
 [0.50328326]
 [0.5031633 ]
 [0.5036161 ]
 [0.5031633 ]
 [0.50330824]
 [0.5033737 ]
 [0.5033737 ]
 [0.5032523 ]
 [0.5031633 ]
 [0.50352484]
 [0.5033737 ]
 [0.50326765]
 [0.50332224]
 [0.5032802 ]]


In [ ]:
def predict_sentiment(text):
  tokenizer = Tokenizer()
  X_tests = tokenizer.texts_to_sequences([text])
  tokenizer.fit_on_texts(texts)
  X_tests = tokenizer.texts_to_sequences(text)
  X_tests = pad_sequences(X_tests,maxlen=max_review_length)

  prediction = int[model.predict(X_tests).round().item()]

  print("Predicted label: ", y_train[1][prediction])

test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)
test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

ValueError: ignored